In [67]:
import collections


first_file_path = "./Fajlovi/1000G_test.vcf"
second_file_path = "./Fajlovi/1000G_test_2.vcf"

merged_file = open("merged.vcf","w+")


#možda da napravim jednu veliku klasu VCF_file u kojoj ću imati metode za sortiranje dictionaries-a filters i formats

class File_format: 
    def __init__(self, line):
        self.line = line
        self.vcf_version = ""
        self.extract_file_format_version()
        
    def __eq__(self, other):
        return self.line == other.line
    
    def extract_file_format_version(self):
        try:
            version = self.line.split("=VCFv",1)[1]
            self.vcf_version = version
        except:
            pass
    

class Filter_header:
    def __init__(self, line):
        self.line = line
        self.id = ""
        self.description = ""
        self.extract_id()
        self.extract_description()
        
    def extract_id(self):
        try:
            id = (self.line.split("ID="))[1].split(",")[0]
            self.id = id
        except:
            pass
    
    def extract_description(self):
        try:
            description = (self.line.split('Description="'))[1].split('"')[0]
            self.description = description
        except:
            pass

# ##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
class Format_header:
    def __init__(self, line):
        self.line = line
        self.id = ""
        self.number = ""
        self.type = ""
        self.description = ""
        self.extract_id()
        self.extract_number()
        self.extract_type()
        self.extract_description()
        
    def extract_id(self):
        try:
            self.id = (self.line.split("ID="))[1].split(",")[0]
        except:
            pass
    
    def extract_number(self):
        try:
            self.number = (self.line.split("Number="))[1].split(",")[0]            
        except:
            pass
        
    def extract_type(self):
        try:
            self.type = (self.line.split("Type="))[1].split(",")[0]            
        except:
            pass
    
    def extract_description(self):
        try:
            self.description = (self.line.split('Description="'))[1].split('"')[0]
        except:
            pass
    
class Filter_lifted_variant:
    def __init__(self, line):
        self.line = line
        #da li nam trebaju sva ona pojedinačna polja
    
    
# ##INFO=<ID=GentrainScore,Number=.,Type=Float,Description="Gentrain Score">
class Info_header:
    def __init__(self, line):
        self.line = line
        self.id = ""
        self.number = ""
        self.type = ""
        self.description = ""
        self.extract_id()
        self.extract_number()
        self.extract_type()
        self.extract_description()
        
    def extract_id(self):
        try:
            self.id = (self.line.split("ID="))[1].split(",")[0]
        except:
            pass
    
    def extract_number(self):
        try:
            self.number = (self.line.split("Number="))[1].split(",")[0]            
        except:
            pass
        
    def extract_type(self):
        try:
            self.type = (self.line.split("Type="))[1].split(",")[0]            
        except:
            pass
    
    def extract_description(self):
        try:
            self.description = (self.line.split('Description="'))[1].split('"')[0]
        except:
            pass
        
# ##contig=<ID=chr13,length=114364328,assembly=20>
class Contig_header:
    def __init__(self, line):
        self.line = line
        self.id = ""
        self.length = ""
        self.assembly = ""
        self.extract_id()
        self.extract_length()
        self.extract_assembly()
        
    def extract_id(self):
        try:
            self.id = (self.line.split("ID="))[1].split(",")[0]
        except:
            pass
    
    def extract_length(self):
        try:
            self.number = (self.line.split("length="))[1].split(",")[0]            
        except:
            pass
        
    def extract_assembly(self):
        try:
            self.type = (self.line.split("assembly="))[1].split(">")[0]            
        except:
            pass

    #do I need to close merged_file ???
    #get rid of filters, GarbageCollector or just filters = {}

        

#pošto se filteri trebaju sortirati trebam kupiti filtere u jednu listu 
#da li ovdje koristiti listu ili nešto drugo, uzećemo dictionary
#kod filtera sortiranje ide po ID 
file_formats = {}
filters = {}
formats = {}
infos = {}
contigs = {}
filter_lifted_variants = list()

#WILDIDEA
#da li napraviti da je izlazni fajl kopija većeg fajla i onda dodavati dodatne linije
   
#ovo moramo napraiti da bude samo jedna funkcija, da može da radi i prvi i drugi fajl, a kasnije i proizvoljan niz fajlova
def read_specific_tag_of_file(tag, list_of_files):
    
    for item in list_of_files:
        previous_position_of_file = item.tell()
        line_of_file = item.readline()
        while(line_of_file.startswith("##" + tag)):
            create_object_add_to_dict(tag, line_of_file)
            previous_position_of_file = item.tell()
            line_of_file = item.readline()
        #rewind file 
        item.seek(previous_position_of_file)
    
    write_tags_in_file(tag)        
    
def write_tags_in_file(tag):
    if(tag == "fileformat"):
        # first sort 
        for key, value in sorted(file_formats.items()):
            merged_file.write(value.line)
    elif(tag == "FILTER"):
        for key, value in sorted(filters.items()):
            merged_file.write(value.line)
    elif(tag == "FORMAT"):
        for key, value in sorted(formats.items()):
            merged_file.write(value.line)
    elif(tag == "FilterLiftedVariants"):
        for value in filter_lifted_variants:
            merged_file.write(value.line) 
    elif(tag == "INFO"):
        for key, value in sorted(infos.items()):
            merged_file.write(value.line)
    elif(tag == "contig"):
        for key, value in sorted(contigs.items()):
            merged_file.write(value.line)
    

def create_object_add_to_dict(tag, line):
    if(tag == "fileformat"):
        file_format = File_format(line)
        file_formats[file_format.vcf_version] = file_format
    elif(tag == "FILTER"):
        filter_header = Filter_header(line)
        if(filter_header not in filters.values()):
            filters[filter_header.id] = filter_header
    elif(tag == "FORMAT"):
        format_header = Format_header(line)
        if(format_header not in formats.values()):
            formats[format_header.id] = format_header  
    elif(tag == "FilterLiftedVariants"):
        filter_lifted_variant = Filter_lifted_variant(line)
        filter_lifted_variants.append(filter_lifted_variant) 
    elif(tag == "INFO"):
        info_header = Info_header(line)
        if(info_header not in infos.values()):
            infos[info_header.id] = info_header
    elif(tag == "contig"):
        contig_header = Contig_header(line)
        if(contig_header not in contigs.values()):
            contigs[contig_header.id] = contig_header 

def get_tag(list_of_files):
    
    diction={}
    
    for item in list_of_files:
        previous_position_of_file = item.tell()
        tag = item.readline().split("=")[0]
        item.seek(previous_position_of_file) 
        diction[tag]=item
    sort_dict = collections.OrderedDict(sorted(diction.items()))
    return list(sort_dict.keys())[0]
   
# context manager
loop=True
with open(first_file_path) as first_file_object:
    with open(second_file_path) as second_file_object:
        list_of_files = [first_file_object,second_file_object]
        while True:
            tag=get_tag(list_of_files)
            if tag.startswith("##"):
                print(tag)
                read_specific_tag_of_file(tag[2:], list_of_files)
            else:
                break
        
    

##fileformat
##FILTER
##FORMAT
##FilterLiftedVariants
##INFO
##contig
##source


In [64]:
import collections 

diction = {
    '##fileformat' : 2,
    '##FILTER' : 4,
    '##FORMAT' : 1,
    '##A' : 3,
    '##b' : 6 
}
sort_dict = collections.OrderedDict(sorted(diction.items()))
print(sort_dict)
list(sort_dict.keys())[0]

OrderedDict([('##A', 3), ('##FILTER', 4), ('##FORMAT', 1), ('##b', 6), ('##fileformat', 2)])


'##A'